In [19]:
import cohere
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
from sklearn.cluster import KMeans
# from bertopic.vectorizers import ClassTFIDF
from sklearn.feature_extraction.text import CountVectorizer

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

api_key = os.getenv('COHERE_API_KEY')
co = cohere.Client(api_key)

In [20]:
#Load the embeddings matrix
embeds = np.load('askhn3k_embeds.npy')

# Load the dataframe containing the text and metadata of the posts
df = pd.read_csv('https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_df.csv', index_col=0)

print(f'Loaded a DataFrame with {len(df)} rows and an embeddings matrix of dimensions {embeds.shape}')

Loaded a DataFrame with 3000 rows and an embeddings matrix of dimensions (3000, 1024)


In [21]:
# Let's glance at the contents of the dataframe with the text and metadata
df.head()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,"I'm a software engineer going blind, how should I prepare?",NaN,"I&#x27;m a 24 y&#x2F;o full stack engineer (I know some of you are rolling your eyes right now, just highlighting that I have experience on frontend apps as well as backend architecture). I&#x27;ve been working professionally for ~7 years building mostly javascript projects but also some PHP. Two years ago I was diagnosed with a condition called &quot;Usher&#x27;s Syndrome&quot; - characterized by hearing loss, balance issues, and progressive vision loss.<p>I know there are blind software engineers out there. My main questions are:<p>- Are there blind frontend engineers?<p>- What kinds of software engineering lend themselves to someone with limited vision? Backend only?<p>- Besides a screen reader, what are some of the best tools for building software with limited vision?<p>- Does your company employ blind engineers? How well does it work? What kind of engineer are they?<p>I&#x27;m really trying to get ahead of this thing and prepare myself as my vision is degrading rather quickly. I&#x27;m not sure what I can do if I can&#x27;t do SE as I don&#x27;t have any formal education in anything. I&#x27;ve worked really hard to get to where I am and don&#x27;t want it to go to waste.<p>Thank you for any input, and stay safe out there!<p>Edit:<p>Thank you all for your links, suggestions, and moral support, I really appreciate it. Since my diagnosis I&#x27;ve slowly developed a crippling anxiety centered around a feeling that I need to figure out the rest of my life before it&#x27;s too late. I know I shouldn&#x27;t think this way but it is hard not to. I&#x27;m very independent and I feel a pressure to &quot;show up.&quot; I will look into these opportunities mentioned and try to get in touch with some more members of the blind engineering community.",NaN,zachrip,3270,1587332026,2020-04-19 21:33:46+00:00,story,22918980,NaN,473.0,NaN,NaN
1,Am I the longest-serving programmer – 57 years and counting?,NaN,"In May of 1963, I started my first full-time job as a computer programmer for Mitchell Engineering Company, a supplier of steel buildings. At Mitchell, I developed programs in Fortran II on an IBM 1620 mostly to improve the efficiency of order processing and fulfillment. Since then, all my jobs for the past 57 years have involved computer programming. I am now a data scientist developing cloud-based big data fraud detection algorithms using machine learning and other advanced analytical technologies. Along the way, I earned a Master’s in Operations Research and a Master’s in Management Science, studied artificial intelligence for 3 years in a Ph.D. program for engineering, and just two years ago I received Graduate Certificates in Big Data Analytics from the schools of business and computer science at a local university (FAU). In addition, I currently hold the designation of Certified Analytics Professional (CAP). At 74, I still have no plans to retire or to stop programming.",NaN,genedangelo,2634,1590890024,2020-05-31 01:53:44+00:00,story,23366546,NaN,531.0,NaN,NaN
2,Is S3 down?,NaN,I&#x27;m getting<p>{\n &quot;errorCode&quot; : &quot;InternalError&quot;\n}<p>When I attempt to use the AWS Console to view s3,NaN,iamdeedubs,2589,1488303958,2017-02-28 17:45:58+00:00,story,13755673,NaN,1055.0,NaN,NaN
3,What tech job would let me get away with the least real work possible?,NaN,"Hey HN,<p>I&#x27;ll probably get a lot of flak for this. Sorry.<p>I&#x27;m an average developer looking for ways to work as little as humanely possible.<p>The pandemic made me realize that I do not care about working anymore. The software I build is useless. Time flies real fast and I have to focus on my passions (which are not monetizable).<p>Unfortunately, I require shelter, calories and hobby materials. Thus the need for some kind of job.<p>Which leads me to ask my fellow tech workers, what kind of job (if any)

In [22]:
# Create the search index, pass the size of embedding
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('askhn.ann')

True

In [23]:
# Choose an example (we'll retrieve others similar to it)
example_id = 50

# Retrieve nearest neighbors
similar_item_ids = search_index.get_nns_by_item(example_id,
                                                10, # Number of results to retrieve
                                                include_distances=True)
# Format and print the text and distances
results = pd.DataFrame(data={'post titles': df.iloc[similar_item_ids[0]]['title'], 
                             'distance': similar_item_ids[1]}).drop(example_id)

print(f"Query post:'{df.iloc[example_id]['title']}'\nNearest neighbors:")
results

Query post:'Pick startups for YC to fund'
Nearest neighbors:


,post titles,distance
731,What should we fund at YC Research?,0.782082
2691,What startups were refused from YC but became successful?,0.832201
1859,Which successful startups were rejected by YC?,0.832643
1965,Did your YC (or other incubator) startup fail? What are you doing now?,0.932350
2910,Who's looking for a cofounder?,0.976471
1603,"Non-VC backed founders, any tips on growth?",0.979501
2599,What are some examples of successful companies rejected by YC?,0.987894
1206,Who's looking for a co-founder?,0.997030
1880,How to raise a seed round for people with no elite connections?,1.007138


In [24]:
query = "What is your most profound life insight?"

# Get the query's embedding
# We'll need to embed the query using the same model that embedded the archive
# so the query and archive are using the same embedding space.
query_embed = co.embed(texts=[query],
                  model="small-20220425", 
                  truncate="RIGHT").embeddings

# Retrieve the nearest neighbors
similar_item_ids = search_index.get_nns_by_vector(query_embed[0],10,
                                                include_distances=True)
# Format & print the results
results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['title'], 
                             'distance': similar_item_ids[1]})
print(f"Query:'{query}'\nNearest neighbors:")
results

CohereAPIError: finetuned model small-20220425 is not valid